# Get-started (Azure version)(WIP)

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed PromptFlow SDK

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create and develop a new promptflow batch run
- Evaluate the batch run with a evaluation flow
- Deploy the flow to a remote endpoint.

**Motivations** - This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it, then deploy it to production.

## 0. Install prompt-flow sdk

In [ ]:
!pip install "promptflow-sdk[azure,builtins]" prompt-flow-tools --extra-index-url https://azuremlsdktestpypi.azureedge.net/promptflow/

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
import json
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
# azure version promptflow apis
import promptflow.azure as pf

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# configure global setting pointing to workpsace ml_client
pf.configure(ml_client)

# create the prompt-flow client pointing to azureml workspace
# client = pf.PFClient(ml_client=ml_client)

runtime = "demo-mir" # TODO remove this after serverless ready

## 1.4 Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `web-classification` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before.

Please go to workspace portal, click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections. Learn more on [connections](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-connections?view=azureml-api-2).

## 2. Create a new batch run

`web-classification` is a flow demonstrating multi-class classification with LLM. Given an url, it will classify the url into one web category with just a few shots, simple summarization and classification prompts.

### Set flow path and input data

In [ ]:
# load flow
flow = "../../flows/standard/web-classification"
data = "../../flows/standard/web-classification/data.jsonl"


### Submit batch run

In [ ]:
# batch run on
base_run = pf.run_batch(
    flow=flow,
    data=data,
    runtime=runtime, # TODO hide this
    connections={},  # TODO introduce this
)
print(base_run)

In [ ]:
pf.stream(base_run)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

In [ ]:
pf.visualize(base_run)

## 3. Evaluate your flow run result
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which use Python or LLM etc., to calculate metrics like accuracy, relevance score.

In this notebook, we use `classification-accuracy-eval` flow to evaluate. This is a flow illustrating how to evaluate the performance of a classification system. It involves comparing each prediction to the groundtruth and assigns a "Correct" or "Incorrect" grade, and aggregating the results to produce metrics such as accuracy, which reflects how good the system is at classifying the data.

In [ ]:
eval_run = pf.run_eval(
    flow = "../../flows/evaluation/classification-accuracy-eval",
    data=data,
    batch_run=base_run,
    inputs_mapping={"groundtruth": "${data.answer}","prediction": "${batch_run.outputs.category}"},
    runtime=runtime,
)


In [ ]:
pf.stream(eval_run)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

In [ ]:
metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([base_run, eval_run])

### Create another batch run with different variant node

In this example, `web-classification`'s node `summarize_text_content` has two variants: `variant_0` and `variant_1`. The difference between them is the inputs parameters:

variant_0:

    - inputs:
        - deployment_name: text-davinci-003
        - max_tokens: '128'
        - temperature: '0.2'
        - text: ${fetch_text_content_from_url.output}

variant_1:

    - inputs:
        - deployment_name: text-davinci-003
        - max_tokens: '256'
        - temperature: '0.3'
        - text: ${fetch_text_content_from_url.output}


You can check the whole flow definistion at [flow.dag.yaml](../../flows/standard/web-classification/flow.dag.yaml)

In [ ]:
# use the variant1 of the summarize_text_content node.
variant_run = pf.run_batch(
    flow=flow,
    data=data,
    variant="${summarize_text_content.variant_1}",  # here we specify node "summarize_text_content" to use variant 1 verison.
    runtime=runtime, # TODO hide this
    connections={},  # TODO introduce this
)


In [ ]:
pf.stream(variant_run)

In [ ]:
details = pf.get_details(variant_run)
details.head(10)

### Run evaluation against variant batch run

In [ ]:
eval_flow = "../../flows/evaluation/classification-accuracy-eval"

eval_run_variant = pf.run_eval(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    batch_run=variant_run,  # use the batch run as the variant
    inputs_mapping={"groundtruth": "${data.answer}","prediction": "${batch_run.outputs.category}"},  # map the url field from the data to the url input of the flow
    runtime=runtime,
)

In [ ]:
pf.stream(eval_run_variant)

In [ ]:
details = pf.get_details(eval_run_variant)
details.head(10)

In [ ]:
metrics = pf.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([eval_run, eval_run_variant])

# Next Steps

Learn more on how to:
- run the flow as a component in a azureml pipeline: [flow in pipeline](../flow-in-pipeline/pipeline.ipynb).